In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm_notebook
import io
import os.path
import glob

In [ ]:
# ensemble

df_test_imgs = pd.read_csv('test_imgs.csv').values[:,[1,2]]
categories = pd.read_csv('category_names.csv')
df_sub = pd.read_csv('sample_submission.csv')
fp = np.memmap('ensemble_all.mm', mode='w+', shape=(1768182,5270))

bar = tqdm_notebook(total=len(df_sub))

r1 = [0.4, 1, 1.5, 2]
r2 = [0.5, 0.5, 1, 1.5]

for j in range(len(df_sub)):

    n_imgs = df_test_imgs[j,1]
    
    a3    = np.load('predict/a3/%d.npy'%j)    
    a3_n1 = np.load('predict/a3_n1/%d.npy'%j)

    if n_imgs == 1:
        a3_x1 = np.load('predict/a3_x1/%d.npy'%j)        
        
        score_a3 = a3_n1[0] + a3_x1[0] + a3[0]
        
    elif n_imgs == 2:
        a3_n6 = np.load('predict/a3_n6/%d.npy'%j)
        a3_x2 = np.load('predict/a3_x2/%d.npy'%j)

        score_a3 = np.multiply(a3_n6[0] + a3_x2[1] + a3[1]*0.1, a3_n1[0] + a3_x2[0] + a3[0]*0.1)**(1/n_imgs)
        
    elif n_imgs == 3:
        a3_n6 = np.load('predict/a3_n6/%d.npy'%j)
        a3_x3 = np.load('predict/a3_x3/%d.npy'%j)

        p1 = a3_n1[0] + a3_x3[0] + a3[0]*0.1
        p2 = a3_n6[0] + a3_x3[1] + a3[1]*0.1
        p3 = a3_n6[1] + a3_x3[2] + a3[2]*0.1
    
        score_a3 = (p1*p2*p3)**(1/n_imgs)
        
    elif n_imgs == 4:
        a3_n6 = np.load('predict/a3_n6/%d.npy'%j)
        a3_x4 = np.load('predict/a3_x4/%d.npy'%j)
    
        p1 = a3_n1[0] + a3_x4[0] + a3[0]*0.1
        p2 = a3_n6[0] + a3_x4[1] + a3[1]*0.1
        p3 = a3_n6[1] + a3_x4[2] + a3[2]*0.1
        p4 = a3_n6[2] + a3_x4[3] + a3[3]*0.1
    
        score_a3 = (p1*p2*p3*p4)**(1/n_imgs)
        
    score_a3 = score_a3/np.sum(score_a3)
    
    # v2
    v2    = np.load('predict/v2/%d.npy'%j)    
    v2_n1 = np.load('predict/v2_n1/%d.npy'%j)

    if n_imgs == 1:
        v2_x1 = np.load('predict/v2_x1/%d.npy'%j)
        
        score_v2 = v2[0] + v2_n1[0]*r1[0] + v2_x1[0]
    else:
        score_v2 = np.multiply(np.prod(v2[1:], axis=0), v2[0]*r1[n_imgs-1]+v2_n1[0])**(1/n_imgs)
        
    score_v2 = score_v2/np.sum(score_v2)
    
    score = (score_a3 + score_v2*r2[n_imgs-1])/(1+r2[n_imgs-1])

    label = categories.loc[np.argmax(score), 'category_id']
    df_sub.set_value(j, 'category_id', label)

    fp[j] = np.clip(score*255,0,255).astype('uint8')
           
    bar.update()
    
df_sub.to_csv('ensemble_all.csv.gz', index=False, compression='gzip')
del fp